In [3]:
import cv2
import pytesseract
import numpy as np

def reorder(myPoints):
    myPoints = myPoints.reshape((4, 2))  
    #print(myPoints)
    myPointsNew = np.zeros((4, 1, 2), np.int32)  
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]  
    myPointsNew[3] = myPoints[np.argmax(add)]  
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] = myPoints[np.argmin(diff)]  
    myPointsNew[2] = myPoints[np.argmax(diff)]  

    return myPointsNew
##################################################################
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
config = ("-l eng --oem 1 --psm 6")
saving_path = 'D:\\Pavan\\Webpages\\Outputs\\google login page_opencv210.png'
path = cv2.imread("D:\\Pavan\\Webpages\\capture.png")
#################################################################
(h,w,d) = path.shape

#resizing image
new_h = 500
new_w = (int)(500*w/h)
new_d=d

path= cv2.resize(path,(new_w,new_h))
cv2.imshow('Website Window',path)
path1 = path.copy()
img = cv2.cvtColor(path,cv2.COLOR_BGR2GRAY)
#cv2.imshow("Gray website window",img)
img1=img.copy()
img2 = img.copy()
img3 = img.copy()

G_blur = cv2.GaussianBlur(img1,(3,3),10)
G_blur1 = cv2.medianBlur(img2,3)
G_blur2 = cv2.bilateralFilter(img2,3,5,10)
#cv2.imshow("Gblur", G_blur)

canny = cv2.Canny(G_blur,1,10)
canny1 = cv2.Canny(G_blur1,1,10)
canny2 = cv2.Canny(G_blur2,1,10)

#cv2.imshow("Canny",canny)
"""
ret,thresh1 = cv2.threshold(G_blur2,250,255,cv2.THRESH_BINARY)
img2, contours, hierarchy2 = cv2.findContours(thresh1,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
"""

img, contours, hie = cv2.findContours(canny,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
img1, contours1, hierarchy1 = cv2.findContours(canny1,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
img2, contours2, hierarchy2 = cv2.findContours(canny2,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

contours = np.append(contours, contours1)
contours = np.append(contours, contours2)

cnt1 = []

for i in range(len(contours2)):
    k = cv2.contourArea(contours[i])
    if(k>500 and k<50000):
        cnt1.append(contours[i])
cnt2 = []       
for i in range(len(cnt1)):
    if hierarchy2[0,i,3]==-1:
        cnt2.append(cnt1[i])
        
imgWarpColored = []

for c in cnt2:
    k=cv2.contourArea(c)
    if k>500 and k<50000:
        peri = cv2.arcLength(c, True)  
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx)==4:
            #print(approx)
            #print(approx[1,0,0])
            approx = reorder(approx)
            #print(approx)
            pts1 = np.float32(approx)
            widthImg = approx[1,0,0] - approx[0,0,0]
            heightImg = approx[2,0,1]-approx[0,0,1]
            pts2 = np.float32([[0, 0], [widthImg, 0],  [0, heightImg], [widthImg, heightImg]]) 
            matrix = cv2.getPerspectiveTransform(pts1, pts2)  
            img = cv2.warpPerspective(path, matrix, (widthImg, heightImg))
            imgWarpColored.append(img)
            ret,thresh1 = cv2.threshold(img,200,255,cv2.THRESH_BINARY)
            img = cv2.resize(thresh1,None,fx=3, fy=3, interpolation = cv2.INTER_CUBIC)
            boxes = pytesseract.image_to_data(img)
            for a,b in enumerate(boxes.splitlines()):
                #print(b)
                if a!=0:
                    b = b.split()
                    #if len(b)==12:
                    x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
                    x=int((x+1)/3)
                    y=int((y+1)/3)
                    w=int((w+1)/3)
                    h=int((h+1)/3)
                    x=x+approx[0,0,0]
                    y=y+approx[0,0,1]
                    if (w*h)<800:
                        cv2.rectangle(path, (x,y), (x+w, y+h), (0, 0, 255), 1)
            cnt = cv2.drawContours(path, c, -1, (0, 255, 0), 2)
cv2.imshow("Gray website window",path)
cv2.imwrite(saving_path,path)
#plt.imshow(path)
#print(len(imgWarpColored))
text = []
for warp in imgWarpColored:
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2RGB)
    #kernel = np.ones((5, 5), np.uint8)
    #warp = cv2.morphologyEx(warp, cv2.MORPH_OPEN, kernel)
    #warp = cv2.threshold(warp, 0, 255, cv2.THRESH_BINARY)[1]
    txt = pytesseract.image_to_string(warp,config= config)
    text.append(txt)
unique_text = []

for t in text:
    if t not in unique_text:
        unique_text.append(t)

for t in unique_text:
    print(t)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

Next

Confirm:
f=
Password
(Username @omailcom
(@gmailcom:
Last name
‘Fost name
Ey
